In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [45]:
lcat_text_path = "resources/images/lightfootcat/images/extracted_text.txt" #"resources/images/hanbury/extracted_text.txt"#
def read_file(file_path):
    """Read the content of a file."""
    with open(file_path, "r") as file:
        return file.read()
    

lcat_text = read_file(lcat_text_path)

print("Hanbury text loaded successfully.")

Hanbury text loaded successfully.


In [46]:
len(lcat_text)

246008

In [47]:
import re

## Cleaning

#### Page numbers and Catalogue common headers

In [48]:
from lib.data_processing.text_processing import TextProcessor

tp = TextProcessor()

In [49]:
divisions = ["Dicotyledones", "Monocotyledones"]

clean_text = tp.preprocess_text(lcat_text, divisions[0])

print(clean_text[:5000])

Dicotyledones

ACERACEAE

Acer campestre L.
1 folder. Acer campestre [TA]
Acer pseudoplatanus L.
2 folders.
Folder 1. Acer Pseudo-Platanus [G]. i. "Maple. Bulls: [Bulstrode] Park" [JL]
Folder 2. Acer Pseudo-Platanus [TA].

AMARANTHACEAE

Amaranthus lividus L., Flora Europaea
1: 110 (1964)
1 folder. Amaranthus Blitum [TA].
i. Cites Ray's Syn. 1957. ii. "Blite Amaranth. Aug.: It is often found on Dunghills in the neighbourhood of London. I gather'd this on a Dunghill at Fulham near London" [JL]. iii. "Amaranthus Blitum. Monoec: 5. and:" [JL]

ARALIACEAE

Hedera helix L.
1 folder. Hedera Helix [TA].

ARISTOLOCHIACEAE

Asarum europaeum L.
1 folder. Asarum europaeum [TA].
i. "Asarum europaeum Lin: Asarabacca. Anglis. ex horto" [JL].

BALSAMINACEAE

Impatiens noli-tangere L.
1 folder. Impatiens Noli me tangere [G].

BETULACEAE

Alnus glutinosa (L.) Gaertn.
1 folder. Betula Alnus [TA].
Betula nana L.
1 folder. Betula nana [G].
CAMpanulaceae
Campanula latifolia L.
2 folders.
Folder 1. Companul

In [50]:
div_struct = tp.split_by_divisions(clean_text, divisions)

In [51]:
div_struct

{'Dicotyledones': '\n\nACERACEAE\n\nAcer campestre L.\n1 folder. Acer campestre [TA]\nAcer pseudoplatanus L.\n2 folders.\nFolder 1. Acer Pseudo-Platanus [G]. i. "Maple. Bulls: [Bulstrode] Park" [JL]\nFolder 2. Acer Pseudo-Platanus [TA].\n\nAMARANTHACEAE\n\nAmaranthus lividus L., Flora Europaea\n1: 110 (1964)\n1 folder. Amaranthus Blitum [TA].\ni. Cites Ray\'s Syn. 1957. ii. "Blite Amaranth. Aug.: It is often found on Dunghills in the neighbourhood of London. I gather\'d this on a Dunghill at Fulham near London" [JL]. iii. "Amaranthus Blitum. Monoec: 5. and:" [JL]\n\nARALIACEAE\n\nHedera helix L.\n1 folder. Hedera Helix [TA].\n\nARISTOLOCHIACEAE\n\nAsarum europaeum L.\n1 folder. Asarum europaeum [TA].\ni. "Asarum europaeum Lin: Asarabacca. Anglis. ex horto" [JL].\n\nBALSAMINACEAE\n\nImpatiens noli-tangere L.\n1 folder. Impatiens Noli me tangere [G].\n\nBETULACEAE\n\nAlnus glutinosa (L.) Gaertn.\n1 folder. Betula Alnus [TA].\nBetula nana L.\n1 folder. Betula nana [G].\nCAMpanulaceae\nCam

In [52]:
struct = dict()
for current_div, div_content in div_struct.items():
    current_div = current_div.strip()
    #print(f"==> Processing {current_div}")


    family_split = tp.split_by_families(div_content)
    struct.setdefault(current_div, family_split)
        

In [77]:
sample = struct["Dicotyledones"][1]

In [78]:
sample_text = sample['text']

In [79]:
print(sample_text)



Amaranthus lividus L., Flora Europaea
1: 110 (1964)
1 folder. Amaranthus Blitum [TA].
i. Cites Ray's Syn. 1957. ii. "Blite Amaranth. Aug.: It is often found on Dunghills in the neighbourhood of London. I gather'd this on a Dunghill at Fulham near London" [JL]. iii. "Amaranthus Blitum. Monoec: 5. and:" [JL]




In [57]:
len(sample_text)

199

In [14]:
overlap = 2000
chunk_size = 4000
chunks = [sample_text[i:i+chunk_size] for i in range(0, len(sample_text), overlap)]

print(len(chunks))

5


In [15]:
prompt = "resources/prompts/hanbury_prompt.yaml" #lightfootcat

from lib.model.ocr_model import OCRModel

ocr_model = OCRModel(prompt)

/mnt/apps/users/ikarunak/conda/envs/dd_lcat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[1007 15:26.07 @file_utils.py:31]  INF  PyTorch version 2.8.0+cu129 available.
[1007 15:26.07 @file_utils.py:69]  INF  Disabling Tensorflow because USE_TORCH is set


2025-10-07 15:26:08,146 - lib.utils.promptLoader - INFO - Loading prompt from resources/prompts/hanbury_prompt.yaml
2025-10-07 15:26:08,205 - lib.utils.promptLoader - INFO - Loading prompt from ./resources/settings.yaml
2025-10-07 15:26:08,214 - lib.model.base_model - INFO - Loading model: mistral7b with batch size: 1, max tokens: 4096, temperature: 0.01
2025-10-07 15:26:10,136 - lib.model - INFO - Importing model: mistral7b
2025-10-07 15:26:10,138 - lib.model - INFO - Model imported successfully: mistral7b


In [16]:
from pprint import pprint

In [17]:
from pydantic import BaseModel, Field
from typing import Optional, List

class Taxon(BaseModel):
    scientific_name: Optional[str] = Field(default=None, description="Scientific name of the taxon with scientific authority if available. Include any indexing where applicable (e.g 1., 2., ....).")
    #index: Optional[str] = Field(default=None, description="Index of the taxon in the list if available. Can be any valid indexing format (e.g 1., 2., a.,b.,...).")
    #part_of_sentence: Optional[bool] = Field(default=None, description="Boolean indicating if the name is part of a sentence (True) or start of a block of information (False).")

class SpeciesResponse(BaseModel):
    taxa: Optional[List[Taxon]]

In [18]:

system_prompt = (
    "You are an expert extraction algorithm tasked with finding scientific species names of plants and fungi from scientific literature.\n"
    "You should extract all scientific species names present in the text.\n"
    "Extract the names verbatim. Do not alter the extracted text by correcting spellings, expanding abbreviations or summarising the text.\n"
    "You should include all scientific authorities and indexing if they appear in the text.\n"
    "No Explanations. Do not include any additional information or context.\n"
    "If you do not know the value of an attribute asked to extract, return null for the attribute value.\n"
    "Output must strictly match this schema.\n"
    "### Schema: \n"
    f"{SpeciesResponse.model_json_schema()}\n"
)

user_prompt = lambda text: (
    f"{text}\n"
)

conversation = lambda text: [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt(text)}]

# sample = family_chunks["Dicotyledones"][8][1][:1000]

# prettyPrint(f"Sample text for analysis:\n{sample}") 

# conversation = [{"role": "system", "content": system_prompt},
#                 {"role": "user", "content": user_prompt(sample)}]


# output = ocr_model.model(conversation=conversation)


# prettyPrint(f"Output from the model:\n{output[0]}")



In [19]:
from tqdm import tqdm

In [20]:
#current_conv = conversation(chunks[0])
conversations = [conversation(chunk) for chunk in chunks]
output = ocr_model.model(conversation=conversations)
pprint(output)

Loading model for [mistralai/Mistral-7B-Instruct-v0.3] to device [cuda]


Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.01s/it]


Loading processor for [mistralai/Mistral-7B-Instruct-v0.3] to device [cuda:0]
2025-10-07 15:26:15,829 - lib.model.hf_models.hf_model - WARNING - Pad token is None. Setting pad token to eos token.


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['[\n'
 '  {\n'
 '    "taxa": [\n'
 '      {\n'
 '        "scientific_name": "Aconitum Napellus, L.",\n'
 '        "title": "Taxon"\n'
 '      },\n'
 '      {\n'
 '        "scientific_name": "Aconitum Lycocotonum, L.",\n'
 '        "title": "Taxon"\n'
 '      },\n'
 '      {\n'
 '        "scientific_name": "Actaea spicata, L.",\n'
 '        "title": "Taxon"\n'
 '      },\n'
 '      {\n'
 '        "scientific_name": "Anemone Hepatica, L.",\n'
 '        "title": "Taxon"\n'
 '      },\n'
 '      {\n'
 '        "scientific_name": "Anemone Pulsatilla, L.",\n'
 '        "title": "Taxon"\n'
 '      },\n'
 '      {\n'
 '        "scientific_name": "Clematis Caripensis, H.B.K.",\n'
 '        "title": "Taxon"\n'
 '      },\n'
 '      {\n'
 '        "scientific_name": "Clematis Flammula, L.",\n'
 '        "title": "Taxon"\n'
 '      },\n'
 '      {\n'
 '        "scientific_name": "Clematis graveolens, Hook.",\n'
 '        "title": "Taxon"\n'
 '      },\n'
 '      {\n'
 '        "scientific_name": 

In [21]:
pprint(chunks[-1])

('imens of stem from the same plant, Jamaica ; N. Wilson, 1867.\n'
 "24. Jateorrhiza palmata, Miers. (Oliver, 'Flora of Tropical Africa,' vol. "
 'i., p. 42.)\n'
 'a. Leaf and flowering spray of male plant, and detached flowers.\n'
 'Botanical Gardens, Mauritius ; J. Horne, 1868.\n'
 'b. Detached leaf of female plant; J. Horne, Botanical Gardens, Mauritius, '
 '1868.\n'
 "Bojer in his 'Hortus Mauritianus' (Maurice, 1837) says of this plant on p. 7 "
 ':-\n'
 '"... a été introduite dans l\'ile par le capitaine W. F. W. Owen et '
 'maintenant cultivée à Bois Chéri et en d\'autre endroits." D.H.\n'
 'c. Leaf bases, one with free and the other with over-lapping basal lobes.\n'
 'Botanical Gardens, Trinidad, 1873.\n'
 'A letter from Mr. H. Prestoe, of the Trinidad Botanical Gardens, states as '
 'follows:\n'
 '"I am now able to inform you that stem and leaves having developed on our '
 'big plant of Cocculus palmatus, I find both the leaf forms you have sent me, '
 'in sketch 7th June,

In [22]:
SpeciesResponse.model_json_schema()

{'$defs': {'Taxon': {'properties': {'scientific_name': {'anyOf': [{'type': 'string'},
      {'type': 'null'}],
     'default': None,
     'description': 'Scientific name of the taxon with scientific authority if available. Include any indexing where applicable (e.g 1., 2., ....).',
     'title': 'Scientific Name'}},
   'title': 'Taxon',
   'type': 'object'}},
 'properties': {'taxa': {'anyOf': [{'items': {'$ref': '#/$defs/Taxon'},
     'type': 'array'},
    {'type': 'null'}],
   'title': 'Taxa'}},
 'required': ['taxa'],
 'title': 'SpeciesResponse',
 'type': 'object'}

In [58]:
import spacy
nlp = spacy.load("en_ner_eco_md")

In [94]:
sample_text = struct["Dicotyledones"][5]["text"]
pprint(sample_text)

('\n'
 '\n'
 'Alnus glutinosa (L.) Gaertn.\n'
 '1 folder. Betula Alnus [TA].\n'
 'Betula nana L.\n'
 '1 folder. Betula nana [G].\n'
 'CAMpanulaceae\n'
 'Campanula latifolia L.\n'
 '2 folders.\n'
 'Folder 1. Companula latifolia [TA].\n'
 'Folder 2. Companula latifolia [G].\n'
 'Campanula patula L.\n'
 '1 folder. Campanula patula [TA].\n'
 'Campanula rapunculus L.\n'
 '2 folders.\n'
 'Folder 1. Campanula rapunculus [TA].\n'
 'Folder 2. Campanula rapunculus [G].\n'
 'Campanula rotundifolia L.\n'
 '1 folder. Campanula rotundifolia [TA]. i. "Near Perth" [JL].\n'
 'Campanula trachelium L.\n'
 '2 folders.\n'
 'Folder 1. Campanula Trachelium [TA]:\n'
 'Folder 2. Campanula Trachelium [G].\n'
 'Jasione montana L.\n'
 '1 folder. Jasione montana [TA].\n'
 'Legousia hybrida (L.) Delarb.\n'
 '1 folder. Campanula hybrida [TA].\n'
 'Lobelia dortmanna L.\n'
 '2 folders.\n'
 'Folder 1. Lobelia Dortmannia [TA]. i. Cites Ray\'s Syn. 287. ii. "Water '
 'Gladiole. July & August. In the Lakes of North Wales 

In [95]:
doc = nlp(sample_text)
ents = doc.ents

In [96]:
import pygbif as gbif
from pprint import pprint

In [97]:
sample_family = struct["Dicotyledones"][0]["family"]

In [100]:
pprint(sample_text)

('\n'
 '\n'
 'Alnus glutinosa (L.) Gaertn.\n'
 '1 folder. Betula Alnus [TA].\n'
 'Betula nana L.\n'
 '1 folder. Betula nana [G].\n'
 'CAMpanulaceae\n'
 'Campanula latifolia L.\n'
 '2 folders.\n'
 'Folder 1. Companula latifolia [TA].\n'
 'Folder 2. Companula latifolia [G].\n'
 'Campanula patula L.\n'
 '1 folder. Campanula patula [TA].\n'
 'Campanula rapunculus L.\n'
 '2 folders.\n'
 'Folder 1. Campanula rapunculus [TA].\n'
 'Folder 2. Campanula rapunculus [G].\n'
 'Campanula rotundifolia L.\n'
 '1 folder. Campanula rotundifolia [TA]. i. "Near Perth" [JL].\n'
 'Campanula trachelium L.\n'
 '2 folders.\n'
 'Folder 1. Campanula Trachelium [TA]:\n'
 'Folder 2. Campanula Trachelium [G].\n'
 'Jasione montana L.\n'
 '1 folder. Jasione montana [TA].\n'
 'Legousia hybrida (L.) Delarb.\n'
 '1 folder. Campanula hybrida [TA].\n'
 'Lobelia dortmanna L.\n'
 '2 folders.\n'
 'Folder 1. Lobelia Dortmannia [TA]. i. Cites Ray\'s Syn. 287. ii. "Water '
 'Gladiole. July & August. In the Lakes of North Wales 

In [101]:
import re

for ent in ents:
    if len(ent.text.split(" ")) <= 1:
        continue
    pprint("="*25)
    pprint(ent.text)
    pprint({idx: sent for idx, sent in enumerate(ent.sents)})
    species_name = ent.text
    pprint(re.findall(rf"^{re.escape(species_name)}", sample_text.strip(), flags=re.MULTILINE))
    
    gbif_res = gbif.name_backbone(name=ent.text.lower(), family=sample_family.lower())
    pprint(gbif_res)

'========================='
'Alnus glutinosa (L.) Gaertn'
{0: 

Alnus glutinosa (L.) Gaertn.
1 folder.}
['Alnus glutinosa (L.) Gaertn']
{'canonicalName': 'Alnus glutinosa',
 'class': 'Magnoliopsida',
 'classKey': 220,
 'confidence': 94,
 'family': 'Betulaceae',
 'familyKey': 4688,
 'genus': 'Alnus',
 'genusKey': 2876099,
 'kingdom': 'Plantae',
 'kingdomKey': 6,
 'matchType': 'EXACT',
 'order': 'Fagales',
 'orderKey': 1354,
 'phylum': 'Tracheophyta',
 'phylumKey': 7707728,
 'rank': 'SPECIES',
 'scientificName': 'Alnus glutinosa (L.) Gaertn.',
 'species': 'Alnus glutinosa',
 'speciesKey': 2876213,
 'status': 'ACCEPTED',
 'usageKey': 2876213}
'========================='
'Betula Alnus [TA]'
{0: Betula Alnus [TA].}
[]
{'canonicalName': 'Betula alnus',
 'class': 'Magnoliopsida',
 'classKey': 220,
 'confidence': 93,
 'family': 'Betulaceae',
 'familyKey': 4688,
 'genus': 'Betula',
 'genusKey': 2875008,
 'kingdom': 'Plantae',
 'kingdomKey': 6,
 'matchType': 'EXACT',
 'order': 'Fagales',
 'order

In [66]:
sample_family

'ACERACEAE'

In [93]:
pprint(ents)

()
